In [ ]:
# 필수 라이브러리 임포트
from typing import Tuple
import matplotlib
import meep as mp
import nlopt
import numpy as np

# matplotlib의 백엔드를 "agg"로 설정하여 이미지를 파일로 저장 가능하게 합니다.
matplotlib.use("agg")
import matplotlib.pyplot as plt
 
# Lorentzian 함수 정의: Lorentzian 파라미터를 사용하여 복소 유전율 프로파일을 계산
def lorentzfunc(p: np.ndarray, x: np.ndarray) -> np.ndarray:
    N = len(p) // 3  # 파라미터가 3개씩 묶인 Lorentzian 항의 개수
    y = np.zeros(len(x))  # 결과값을 저장할 배열
    for n in range(N):
        A_n = p[3 * n + 0]  # 첫 번째 파라미터 (진폭)
        x_n = p[3 * n + 1]  # 두 번째 파라미터 (중심 주파수)
        g_n = p[3 * n + 2]  # 세 번째 파라미터 (감쇠율)
        y = y + A_n / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수 계산
    return y

# 잔차 함수 정의: 실제 값과 Lorentzian 모델 간의 차이를 계산하고 그라디언트도 반환
def lorentzerr(p: np.ndarray, x: np.ndarray, y: np.ndarray, grad: np.ndarray) -> float:
    N = len(p) // 3  # Lorentzian 항의 개수
    yp = lorentzfunc(p, x)  # 예측된 유전율 프로파일 계산
    val = np.sum(np.square(abs(y - yp)))  # 실제 값과 예측 값의 차이(L2 norm)

    # 그라디언트 계산
    for n in range(N):
        A_n = p[3 * n + 0]
        x_n = p[3 * n + 1]
        g_n = p[3 * n + 2]
        d = 1 / (np.square(x_n) - np.square(x) - 1j * x * g_n)  # Lorentzian 함수의 도함수
        if grad.size > 0:
            grad[3 * n + 0] = 2 * np.real(np.dot(np.conj(yp - y), d))  # 진폭에 대한 그라디언트
            grad[3 * n + 1] = -4 * x_n * A_n * np.real(np.dot(np.conj(yp - y), np.square(d)))  # 중심 주파수에 대한 그라디언트
            grad[3 * n + 2] = -2 * A_n * np.imag(np.dot(np.conj(yp - y), x * np.square(d)))  # 감쇠율에 대한 그라디언트
    return val  # 최종 오차 반환

# Lorentzian 파라미터 최적화 함수 정의
def lorentzfit(
    p0: np.ndarray,
    x: np.ndarray,
    y: np.ndarray,
    alg=nlopt.LD_LBFGS,
    tol: float = 1e-25,
    maxeval: float = 10000,
) -> Tuple[np.ndarray, float]:
    # NLopt 최적화 설정
    opt = nlopt.opt(alg, len(p0))  # 최적화 알고리즘 설정 (LD_LBFGS는 LBFGS 알고리즘)
    opt.set_ftol_rel(tol)  # 상대 오차 허용 범위 설정
    opt.set_maxeval(maxeval)  # 최대 반복 횟수 설정
    opt.set_lower_bounds(np.zeros(len(p0)))  # 하한값 설정
    opt.set_upper_bounds(float("inf") * np.ones(len(p0)))  # 상한값 설정
    opt.set_min_objective(lambda p, grad: lorentzerr(p, x, y, grad))  # 목표 함수 설정
    local_opt = nlopt.opt(nlopt.LD_LBFGS, len(p0))  # 로컬 최적화 설정
    local_opt.set_ftol_rel(1e-10)  # 로컬 최적화 상대 오차 설정
    local_opt.set_xtol_rel(1e-8)  # 로컬 최적화 X 변화 상대 오차 설정
    opt.set_local_optimizer(local_opt)  # 로컬 최적화 설정 추가
    popt = opt.optimize(p0)  # 최적화 실행
    minf = opt.last_optimum_value()  # 마지막 최적화 값 반환
    return popt, minf  # 최적화된 파라미터와 최소 오차 반환

------------------------------------------------------Al_Palik------------------------------------------------------

In [19]:
# 메인 코드 실행 부분
# SiN.csv 파일에서 복소 유전율 데이터를 불러옴 (wavelength, real(n), imag(n) 순)
mydata = np.genfromtxt("Al_palik_data.csv", delimiter=",")
n = mydata[:, 1] + 1j * mydata[:, 2]  # 복소수 형태로 유전율 데이터 생성

# 유전율의 무한 주파수에서의 값 설정 (eps_inf)
eps_inf = 1
eps = np.square(n) - eps_inf  # 복소 유전율 프로파일 계산

# 주어진 파장 범위로 데이터를 필터링
wl = mydata[:, 0]*1e6
wl_min = 0.399  # 최소 파장 (단위: nm)
wl_max = 0.701  # 최대 파장 (단위: nm)

start_idx = np.where(wl > wl_min)[0][0]  # 최소 파장 이상인 인덱스 찾기

end_idx = np.where(wl < wl_max)[0][-1] + 1  # 최대 파장 이하인 인덱스 찾기

# 주파수(f)로 변환 (주파수는 1/파장으로 계산)
freqs = 1 / wl  # 단위: 1/μm

freqs_reduced = freqs[start_idx:end_idx]  # 필터링된 주파수
wl_reduced = wl[start_idx:end_idx]          # 필터링된 파장 (µm)
eps_reduced = eps[start_idx:end_idx]          # 필터링된 유전율

In [58]:
fit_wl_min = 0.38  # µm
fit_wl_max = 0.73  # µm

fit_idx = np.where((wl_reduced >= fit_wl_min) & (wl_reduced <= fit_wl_max))[0]
freqs_fit = freqs_reduced[fit_idx]
wl_fit = wl_reduced[fit_idx]
eps_fit = eps_reduced[fit_idx]

# ---------------------------
# Lorentzian 피팅 최적화 (지정 구간만 사용)
# ---------------------------
num_lorentzians = 2
# np.random.seed(5)
num_repeat = 50
ps = np.zeros((num_repeat, 3 * num_lorentzians))
mins = np.zeros(num_repeat)

for m in range(num_repeat):
    # 3*num_lorentzians 개의 랜덤 초기값 생성 (10**(랜덤값))
    p_rand = [10 ** (np.random.random()) for _ in range(3 * num_lorentzians)]
    ps[m, :], mins[m] = lorentzfit(p_rand, freqs_fit, eps_fit, nlopt.LD_MMA, 1e-25, 50000)
    print(f"iteration: {m:3d}, error: {mins[m]:.6f}")

# 최적화된 파라미터 선택
idx_opt = np.where(np.min(mins) == mins)[0][0]
print(f"optimal: {mins[idx_opt]:.6f}")

# 최적화된 파라미터로 'Medium' 객체 생성 (Drude 또는 Lorentzian 항 적용)
E_susceptibilities = []
for n_idx in range(num_lorentzians):
    mymaterial_freq = ps[idx_opt][3 * n_idx + 1]
    mymaterial_gamma = ps[idx_opt][3 * n_idx + 2]
    if mymaterial_freq == 0:
        mymaterial_sigma = ps[idx_opt][3 * n_idx + 0]
        E_susceptibilities.append(mp.DrudeSusceptibility(frequency=1.0, gamma=mymaterial_gamma, sigma=mymaterial_sigma))
    else:
        mymaterial_sigma = ps[idx_opt][3 * n_idx + 0] / mymaterial_freq**2
        E_susceptibilities.append(mp.LorentzianSusceptibility(frequency=mymaterial_freq, gamma=mymaterial_gamma, sigma=mymaterial_sigma))
mymaterial = mp.Medium(epsilon=eps_inf, E_susceptibilities=E_susceptibilities)

# ---------------------------
# 피팅 구간(fitting region)에 대해 meep model 계산
# ---------------------------
mymaterial_eps = [mymaterial.epsilon(f)[0][0] for f in freqs_fit]

# ---------------------------
# 그래프 출력 (비교 플롯)
# ---------------------------
plt.close('all')
fig, ax = plt.subplots(ncols=2, figsize=(8,3))

# 왼쪽 그래프: Real(ε)
ax[0].plot(wl_fit, np.real(eps_fit) + eps_inf, 'gs', markersize=4, label="Material data")
ax[0].plot(wl_fit, np.real(mymaterial_eps), 'b-', label="Meep model")
ax[0].set_xlabel("wavelength (µm)")
ax[0].set_ylabel(r"Re($\epsilon$)")
ax[0].legend()
ax[0].grid(True)
ax[0].set_xlim([fit_wl_min+0.02, fit_wl_max-0.03])  # X축 범위를 지정 구간으로 제한

# 오른쪽 그래프: Imag(ε)
ax[1].plot(wl_fit, np.imag(eps_fit), 'gs', markersize=4, label="Material data")
ax[1].plot(wl_fit, np.imag(mymaterial_eps), 'b-', label="Meep model")
ax[1].set_xlabel("wavelength (µm)")
ax[1].set_ylabel(r"Im($\epsilon$)")
ax[1].legend()
ax[1].grid(True)
ax[1].set_xlim([fit_wl_min+0.02, fit_wl_max-0.03])

fig.suptitle("Comparison of Material Data and FDTD Model\n(using Drude-Lorentzian Susceptibility)", fontsize=9)
fig.subplots_adjust(wspace=0.3)
fig.savefig("eps_fit_sample.png", dpi=150, bbox_inches="tight")

iteration:   0, error: 102.892827
iteration:   1, error: 77519.145514
iteration:   2, error: 102.892827
iteration:   3, error: 102.892827
iteration:   4, error: 77519.145514
iteration:   5, error: 102.892827
iteration:   6, error: 102.892827
iteration:   7, error: 7.537164
iteration:   8, error: 102.892827
iteration:   9, error: 102.892827
iteration:  10, error: 7.536368
iteration:  11, error: 102.892827
iteration:  12, error: 87.924780
iteration:  13, error: 7.536082
iteration:  14, error: 7.535769
iteration:  15, error: 87.924780
iteration:  16, error: 102.892827
iteration:  17, error: 77519.145514
iteration:  18, error: 77519.145514
iteration:  19, error: 102.892827
iteration:  20, error: 102.892827
iteration:  21, error: 102.892827
iteration:  22, error: 102.892827
iteration:  23, error: 102.892827
iteration:  24, error: 102.892827
iteration:  25, error: 102.892827
iteration:  26, error: 77519.145514
iteration:  27, error: 87.924780
iteration:  28, error: 7.535769
iteration:  29, e

In [49]:
# CSV 파일의 header: wavelength_um, n_fdtd, k_fdtd
csv_compare_file = "Al_palik_data_FDTD.csv"  # 파일 경로 (원하는 경로로 수정)
csv_data = np.genfromtxt(csv_compare_file, delimiter=",", skip_header=1)
csv_wl = csv_data[:, 0]           # wavelength (µm)
csv_n = csv_data[:, 1] + 1j*csv_data[:, 2]  # 복소 굴절률 n+ik

# 기존 코드와 동일하게, material data에서 eps = n^2 - eps_inf를 사용했으므로
# CSV 데이터도 같은 방식으로 계산하여 실제 유전율 값을 구함
csv_eps = np.square(csv_n) - eps_inf

# 플롯에 CSV 비교 데이터 추가 (실제 ε 값는 Material data와 동일하게 표현)
ax[0].plot(csv_wl, np.real(csv_eps) + eps_inf, 'r--', markersize=6, label="FDTD model")
ax[1].plot(csv_wl, np.imag(csv_eps), 'r--', markersize=6, label="FDTD model")

# 범례 업데이트
ax[0].legend()
ax[1].legend()

# 플롯 저장 (비교 데이터 포함)
fig.savefig("eps_fit_sample_with_csv.png", dpi=150, bbox_inches="tight")
print(f"CSV 비교 데이터가 추가된 플롯이 'eps_fit_sample_with_csv.png'로 저장되었습니다.")

CSV 비교 데이터가 추가된 플롯이 'eps_fit_sample_with_csv.png'로 저장되었습니다.


In [22]:
# wl_fit, Material data (Re, Im), Meep model (Re, Im)
material_Re = np.real(eps_fit) + eps_inf
material_Im = np.imag(eps_fit)
meep_Re     = np.real(mymaterial_eps)
meep_Im     = np.imag(mymaterial_eps)

# 데이터 행렬 생성: 각 행은 [wavelength, Material_Re, Material_Im, Meep_Re, Meep_Im]
data_to_save = np.column_stack((wl_fit, material_Re, material_Im, meep_Re, meep_Im))

# CSV 파일 경로 지정
csv_save_path = "eps_fit_sample_data.csv"

# np.savetxt를 사용하여 CSV 파일로 저장 (header와 함께)
np.savetxt(csv_save_path, data_to_save, delimiter=",",
           header="wavelength(um),Material_Re,Material_Im,Meep_Re,Meep_Im", comments="")

print(f"CSV 파일로 저장 완료: {csv_save_path}")

CSV 파일로 저장 완료: eps_fit_sample_data.csv


In [68]:
# ---------------------------
# 0) 공통 x축(0.4 ~ 0.7 µm) 생성
# ---------------------------
common_x = np.linspace(0.4, 0.7, 30)  # 200점으로 균일 분할

# ---------------------------
# 1) Meep model 데이터 보간 (복소수의 실수/허수부 각각 보간)
# ---------------------------
meep_real_interp = np.interp(common_x, wl_fit, np.real(mymaterial_eps))
meep_imag_interp = np.interp(common_x, wl_fit, np.imag(mymaterial_eps))
meep_interp = meep_real_interp + 1j * meep_imag_interp

# ---------------------------
# 2) CSV로부터 읽은 FDTD 데이터 보간 (복소수 보간)
# ---------------------------
csv_real_interp = np.interp(common_x, csv_wl, np.real(csv_eps))
csv_imag_interp = np.interp(common_x, csv_wl, np.imag(csv_eps))
csv_interp = csv_real_interp + 1j * csv_imag_interp

# ---------------------------
# 3) 상대 오차 계산
#    Relative Error (%) = 100 * |Meep - FDTD| / |FDTD|
# ---------------------------
relative_error = 100.0 * np.abs(meep_interp - csv_interp) / np.abs(csv_interp)

# ---------------------------
# 4) 상대 오차 플롯
# ---------------------------
plt.figure(figsize=(6,4))
plt.plot(common_x, relative_error, 'ko', markersize=4, label="Relative Error (%)")
plt.xlabel("Wavelength (µm)")
plt.ylabel("Relative Error (%)")
plt.title("Relative Error: Meep vs FDTD (Interpolated)")
plt.grid(True, which='both', linestyle='--', alpha=0.7)
plt.legend()
plt.tight_layout()
plt.savefig("relative_error_interpolated.png", dpi=150, bbox_inches='tight')

plt.show()


/tmp/ipykernel_19402/3753918210.py:39: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
